In [372]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [373]:
import sys
sys.path.append('/gdrive/My Drive/2nd Sem/IBM/Covid Project/Code/Scraping Twitter Data')

In [374]:
import private
import re
import tweepy
import difflib
import pandas as pd
import numpy as np
import pickle
import copy
import matplotlib.pyplot as plt
from datetime import date

In [375]:
auth = tweepy.OAuthHandler(private.CONSUMER_KEY, private.CONSUMER_SECRET)
auth.set_access_token(private.OAUTH_TOKEN, private.OAUTH_TOKEN_SECRET)
api = tweepy.API(auth)

#Extracting Covid Tweets

## Twitter Scraped Features Description


![alt text](https://drive.google.com/uc?id=1LAjmD-RIR-oswQ0ocKaXK708Z7thsrAs)









In [376]:
def get_list_of_user_mentions(tweet):
    user_handles = []

    try:
            
        mentions = tweet._json['entities']['user_mentions']
        for item in mentions:
            user_handles.append(item['screen_name'])
        return user_handles

    except:
        return user_handles

    

In [377]:
def get_list_of_photos(tweet):
    photos_list = []

    try:
            
        photos = tweet._json['entities']['media']
        for item in photos:
            photos_list.append(item['media_url_https'])
        return photos_list
    
    except:
        return photos_list

In [378]:
def get_replies_count(tweet,user_handle):
    
    try:
            
        replies=[] 
        q = 'to:'+user_handle
        for full_tweets in tweepy.Cursor(api.user_timeline,screen_name=user_handle,timeout=999999).items(10):
            for tweet in tweepy.Cursor(api.search,q, since_id=tweet_id, result_type='recent',tweet_mode='extended',timeout=999999).items(1000):
                if hasattr(tweet, 'in_reply_to_status_id_str'):
                    if (tweet.in_reply_to_status_id_str==full_tweets.id_str):
                        replies.append(tweet.full_text)

        #Removing Duplicate Replies
        replies = list(dict.fromkeys(replies))
        return len(replies)

    except:
        return 0

In [379]:
def get_all_hashtags(tweet):
    hashtags_list = []

    hashtags = tweet._json['entities']['hashtags']
    for item in hashtags:
        hashtags_list.append(item['text'])


    return hashtags_list

In [380]:
def get_retweet_id(tweet):
    try:
        retweet_id = tweet._json['retweeted_status']['id']
        return retweet_id
    except:
        return 0

In [381]:
def get_retweet_date(tweet):
    try:
        retweet_date = tweet._json['retweeted_status']['created_at']
        return retweet_date
    except:
        return ""

In [382]:
def get_retweet_user_id(tweet):
    try:
        retweet_user_id = tweet._json['retweeted_status']['user']['id']
        return retweet_user_id
    except:
        return ""

In [383]:
def check_tweet_exists(test_tweet, texts_list):
    counter = 1
    test_tweet     = test_tweet.split()

    for existing_tweet in texts_list:
        existing_tweet = existing_tweet.split()
        n              = len(existing_tweet)
        m              = 0
        for word in test_tweet:
            if word in existing_tweet:
                m = m + 1
                if(m/n > 0.5):                                             ### If the new tweet is 50% similar with existing tweet then don't append it
                    #print("Helllo this tweet has been rejected") 
                    return True
        # print("This tweet has passed test number ",counter)
        counter = counter + 1
    return False



In [384]:
### References --> https://towardsdatascience.com/selenium-tweepy-to-scrap-tweets-from-tweeter-and-analysing-sentiments-1804db3478ac

def get_tweets(query, desired_lang = 'hi',maxTweets = 10, tweetsPerQry=10): 
        # empty list to store parsed tweets 
        tweets = [] 
        texts_list = []
        sinceId = None
        max_id = -1
        tweetCount = 0
        
        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = api.search(q=query, lang= desired_lang, tweet_mode= 'extended',count=tweetsPerQry)
                    else:
                        new_tweets = api.search(q=query, lang = desired_lang, tweet_mode= 'extended',count=tweetsPerQry,
                                                since_id=sinceId)
                else:
                    if (not sinceId):
                        new_tweets = api.search(q=query, lang = desired_lang, tweet_mode= 'extended', count=tweetsPerQry,
                                                max_id=str(max_id - 1))
                    else:
                        new_tweets = api.search(q=query, lang = desired_lang, tweet_mode= 'extended', count=tweetsPerQry,
                                                max_id=str(max_id - 1),
                                                since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break
                    
                for tweet in new_tweets:

                    parsed_tweet = {} 

                    # Making Sure that the tweets don't have more than 50% word similarity

                    already_existing_tweet_flag = check_tweet_exists(tweet.full_text,texts_list)
                    if(already_existing_tweet_flag):
                        continue
                    else:
                        tweets.append(parsed_tweet)
                        texts_list.append(tweet.full_text)


                    parsed_tweet['id']                     = tweet.id
                    parsed_tweet['conversation_id']        = tweet.id_str
                    parsed_tweet['created_at']             = tweet.created_at
                    parsed_tweet['date']                   = str(tweet.created_at).split()[0]
                    parsed_tweet['time']                   = str(tweet.created_at).split()[1]
                    parsed_tweet['time_zone']              = "UTC" 
                    parsed_tweet['user_id']                = tweet.user.id
                    parsed_tweet['username']               = tweet.user.screen_name
                    parsed_tweet['name']                   = tweet.user.name
                    parsed_tweet['place']                  = tweet.user.location
                    parsed_tweet['tweet']                  = tweet.full_text
                    parsed_tweet['mentions']               = get_list_of_user_mentions(tweet)
                    parsed_tweet['urls']                   = tweet._json['entities']['urls']
                    parsed_tweet['photos']                 = get_list_of_photos(tweet)
                    parsed_tweet['replies_count']          = 0 #get_replies_count(tweet,tweet.user.screen_name)
                    parsed_tweet['retweet_count']          = tweet.retweet_count
                    parsed_tweet['likes_count']            = tweet.favorite_count
                    parsed_tweet['hashtags']               = get_all_hashtags(tweet)
                    parsed_tweet['cashtags']               = []
                    parsed_tweet['link']                   = "http://twitter.com/anyuser/status/" + tweet.id_str
                    parsed_tweet['retweet']                = tweet.retweeted
                    parsed_tweet['quote_url']              = " "
                    parsed_tweet['video']                  = 0
                    parsed_tweet['geo']                    = tweet._json['geo']
                    parsed_tweet['source']                 = tweet._json['source']
                    parsed_tweet['rt_user_id']             = get_retweet_user_id(tweet)
                    parsed_tweet['retweet_date']           = get_retweet_date(tweet)
                    parsed_tweet['retweet_id']             = get_retweet_id(tweet)
                                     
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id

            except tweepy.TweepError as e:
                print("Tweepy error : " + str(e))
                break
        
        return pd.DataFrame(tweets)

In [385]:
df = get_tweets("covid",'bn',maxTweets=20,tweetsPerQry=20)
df

Downloaded 20 tweets


,id,conversation_id,created_at,date,time,time_zone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweet_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,geo,source,rt_user_id,retweet_date,retweet_id
0,1277587874100068355,1277587874100068355,2020-06-29 13:01:00,2020-06-29,13:01:00,UTC,1249580478950076418,PranenduBhatta1,Pranendu Bhattacharjee,"Tripura, India","RT @PIBAgartala: #COVID_19 ২৯ জুন, ২০২০ সকাল ৮...",[PIBAgartala],[],[],0,41,0,"[COVID_19, CoronawatchIndia, IndiaFightsCorona]",[],http://twitter.com/anyuser/status/127758787410...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",902067067104354306,Mon Jun 29 06:02:58 +0000 2020,1277482672231608321
1,1277585827514777600,1277585827514777600,2020-06-29 12:52:52,2020-06-29,12:52:52,UTC,1257296248379305984,KalipadaDandap3,Kalipada Dandapat,"Jhargram, India",RT @Asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,[Asishsahoo12341],[],[],0,19,0,[],[],http://twitter.com/anyuser/status/127758582751...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",1255000189439545344,Sun Jun 28 16:08:49 +0000 2020,1277272752676732928
2,1277580484164243456,1277580484164243456,2020-06-29 12:31:38,2020-06-29,12:31:38,UTC,861548391972700161,titomukherjee15,Subhayan Mukherjee Tito,"Serampore, West Bengal, India",@Dharmayuddhaa @MamataOfficial মাথামোটা! তোর ব...,"[Dharmayuddhaa, MamataOfficial]","[{'url': 'https://t.co/kmWXRWGgdc', 'expanded_...",[],0,0,0,[],[],http://twitter.com/anyuser/status/127758048416...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",,,0
3,1277578556596383748,1277578556596383748,2020-06-29 12:23:58,2020-06-29,12:23:58,UTC,795241907597123584,aajkaalofficial,Aajkaal Official,"Kolkata, India",১৫ জুলাই অবধি বিশেষ ট্রেন চলাচল বন্ধ তামিলনাড়...,[],"[{'url': 'https://t.co/BtMrFIipeV', 'expanded_...",[https://pbs.twimg.com/media/Ebre6x-UcAAFQTn.jpg],0,0,0,[],[],http://twitter.com/anyuser/status/127757855659...,False,,0,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",,,0
4,1277578344754630662,1277578344754630662,2020-06-29 12:23:08,2020-06-29,12:23:08,UTC,1192400038216916992,HT_Bangla,Hindustan Times Bangla,,করোনা থেকে সেরে ওঠা ব্যক্তিদের সরকারি হাসপাতাল...,[],"[{'url': 'https://t.co/XuHxFkcmas', 'expanded_...",[],0,0,0,[],[],http://twitter.com/anyuser/status/127757834475...,False,,0,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",,,0
5,1277578048166977536,1277578048166977536,2020-06-29 12:21:57,2020-06-29,12:21:57,UTC,1058272790745206784,BharadwajAngira,Angira Bharadwaj,Guwa,"দিল্লীৰ নিচিনা ঠাইত শুনিব পোৱা মতে, #COVID পৰী...",[],"[{'url': 'https://t.co/UUnyW7wwoJ', 'expanded_...",[],0,0,2,[COVID],[],http://twitter.com/anyuser/status/127757804816...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",,,0
6,1277578021000450048,1277578021000450048,2020-06-29 12:21:51,2020-06-29,12:21:51,UTC,89647717,IamArijitRoy,ARIJIT KR. ROY🇮🇳😷,"Asansol, West Bengal, India","COVID যোদ্ধাদের কুর্নিশ রাজ্য সরকারের, ১ জুলাই...",[],"[{'url': 'https://t.co/7wUjKwMVdB', 'expanded_...",[],0,0,0,[],[],http://twitter.com/anyuser/status/127757802100...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",,,0
7,1277577535216250880,1277577535216250880,2020-06-29 12:19:55,2020-06-29,12:19:55,UTC,1272195565632262144,AdvAmitavaAich,অমিতাভ || Amitava,"Gobardanga, West Bengal",এবার স্বাস্থ্য পরিসেবায় সরাসরি অংশগ্রহণ করবেন...,"[MamataOfficial, abhishekaitc, FirhadHakim, it...",[],[https://pbs.twimg.com/ext_tw_video_thumb/1277...,0,0,1,[],[],http://twitter.com/anyuser/status/127757753521...,False,,0,None,"<a href=""http://twitter.com/download/android"" ...",,,0
8,1277575323387715584,1277575323387715584,2020-06-29 12:11:07,2020-06-29,12:11:07,UTC,2241832560,pratidinasomiya,asomiyapratidin,Guwahati,মৰিগাঁৱত পেট্ৰল পাম্পৰ কৰ্মচাৰীৰ দেহত কোৰোনা ভ...,[],"[{'url': 'https://t.co/M6UUkhUET5', 'expanded_...",[],0,0,1,[],[],http://twitter.com/anyuser/status/127757532338...,False,,0,Non

In [386]:
df.to_csv("/gdrive/My Drive/2nd Sem/IBM/Covid Project/Dataset/Scraping Tweets Demo/covid_tweets_raw_features_dump.csv",index=False)

#Extracting User Features

## User Features

![alt text](https://drive.google.com/uc?id=1iXpdyY9eQw-HN7knBKdfhOhf4SPDqPlr)

In [387]:
df = pd.read_csv("/gdrive/My Drive/2nd Sem/IBM/Covid Project/Dataset/Scraping Tweets Demo/covid_tweets_raw_features_dump.csv")
df

,id,conversation_id,created_at,date,time,time_zone,user_id,username,name,place,tweet,mentions,urls,photos,replies_count,retweet_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,geo,source,rt_user_id,retweet_date,retweet_id
0,1277587874100068355,1277587874100068355,2020-06-29 13:01:00,2020-06-29,13:01:00,UTC,1249580478950076418,PranenduBhatta1,Pranendu Bhattacharjee,"Tripura, India","RT @PIBAgartala: #COVID_19 ২৯ জুন, ২০২০ সকাল ৮...",['PIBAgartala'],[],[],0,41,0,"['COVID_19', 'CoronawatchIndia', 'IndiaFightsC...",[],http://twitter.com/anyuser/status/127758787410...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",9.020671e+17,Mon Jun 29 06:02:58 +0000 2020,1277482672231608321
1,1277585827514777600,1277585827514777600,2020-06-29 12:52:52,2020-06-29,12:52:52,UTC,1257296248379305984,KalipadaDandap3,Kalipada Dandapat,"Jhargram, India",RT @Asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,['Asishsahoo12341'],[],[],0,19,0,[],[],http://twitter.com/anyuser/status/127758582751...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",1.255000e+18,Sun Jun 28 16:08:49 +0000 2020,1277272752676732928
2,1277580484164243456,1277580484164243456,2020-06-29 12:31:38,2020-06-29,12:31:38,UTC,861548391972700161,titomukherjee15,Subhayan Mukherjee Tito,"Serampore, West Bengal, India",@Dharmayuddhaa @MamataOfficial মাথামোটা! তোর ব...,"['Dharmayuddhaa', 'MamataOfficial']","[{'url': 'https://t.co/kmWXRWGgdc', 'expanded_...",[],0,0,0,[],[],http://twitter.com/anyuser/status/127758048416...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,0
3,1277578556596383748,1277578556596383748,2020-06-29 12:23:58,2020-06-29,12:23:58,UTC,795241907597123584,aajkaalofficial,Aajkaal Official,"Kolkata, India",১৫ জুলাই অবধি বিশেষ ট্রেন চলাচল বন্ধ তামিলনাড়...,[],"[{'url': 'https://t.co/BtMrFIipeV', 'expanded_...",['https://pbs.twimg.com/media/Ebre6x-UcAAFQTn....,0,0,0,[],[],http://twitter.com/anyuser/status/127757855659...,False,,0,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,0
4,1277578344754630662,1277578344754630662,2020-06-29 12:23:08,2020-06-29,12:23:08,UTC,1192400038216916992,HT_Bangla,Hindustan Times Bangla,NaN,করোনা থেকে সেরে ওঠা ব্যক্তিদের সরকারি হাসপাতাল...,[],"[{'url': 'https://t.co/XuHxFkcmas', 'expanded_...",[],0,0,0,[],[],http://twitter.com/anyuser/status/127757834475...,False,,0,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,0
5,1277578048166977536,1277578048166977536,2020-06-29 12:21:57,2020-06-29,12:21:57,UTC,1058272790745206784,BharadwajAngira,Angira Bharadwaj,Guwa,"দিল্লীৰ নিচিনা ঠাইত শুনিব পোৱা মতে, #COVID পৰী...",[],"[{'url': 'https://t.co/UUnyW7wwoJ', 'expanded_...",[],0,0,2,['COVID'],[],http://twitter.com/anyuser/status/127757804816...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,0
6,1277578021000450048,1277578021000450048,2020-06-29 12:21:51,2020-06-29,12:21:51,UTC,89647717,IamArijitRoy,ARIJIT KR. ROY🇮🇳😷,"Asansol, West Bengal, India","COVID যোদ্ধাদের কুর্নিশ রাজ্য সরকারের, ১ জুলাই...",[],"[{'url': 'https://t.co/7wUjKwMVdB', 'expanded_...",[],0,0,0,[],[],http://twitter.com/anyuser/status/127757802100...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,0
7,1277577535216250880,1277577535216250880,2020-06-29 12:19:55,2020-06-29,12:19:55,UTC,1272195565632262144,AdvAmitavaAich,অমিতাভ || Amitava,"Gobardanga, West Bengal",এবার স্বাস্থ্য পরিসেবায় সরাসরি অংশগ্রহণ করবেন...,"['MamataOfficial', 'abhishekaitc', 'FirhadHaki...",[],['https://pbs.twimg.com/ext_tw_video_thumb/127...,0,0,1,[],[],http://twitter.com/anyuser/status/127757753521...,False,,0,NaN,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,0
8,1277575323387715584,1277575323387715584,2020-06-29 12:11:07,2020-06-29,12:11:07,UTC,2241832560,pratidinasomiya,asomiyapratidin,Guwahati,মৰিগাঁৱত পেট্ৰল পাম্পৰ কৰ্মচাৰীৰ দেহত কোৰোনা ভ...,[],"[{'url': 'https://t.co/M6UUkhUET5', 'expanded_...",[],0,0,1,[],[],http://twitter.com/anyuser/statu

In [388]:
def calculate_days_past(tweet_time):
    #Calculating Dates
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    a1 = int(tweet_time[-1])
    b1 = int(months.index(tweet_time[1])+1)
    c1 = int(tweet_time[2])

    #Todays date
    today = date.today()
    life  =  str(date(today.year,today.month,today.day) - date(a1,b1,c1))
    if(len(life)>7):
        life  = life.split(" ")
        life  = life[0]
        life  = int(life)
        return life
    else:
        return 0

In [389]:
def get_user_info(tweet):
    
    tweet_text   = tweet.full_text
    real_name    = tweet._json['user']['name']
    user_handle  = tweet._json['user']['screen_name']
    tweet_id     = tweet._json['id']
    user_id      = tweet._json['user']['id']
    
    user_info = api.get_user(user_id,user_mode='extended')
    desc = user_info._json['description']
    
    try:
        expanded_url = user_info._json['entities']['url']['urls'][0]['expanded_url']
        expanded_url = 1
    except:
        expanded_url = 0

    
    tweet_link             = "http://twitter.com/anyuser/status/" + str(tweet.id)
    chars_in_desc          = len(desc)
    chars_in_real_name     = len(real_name)
    chars_in_user_handle   = len(user_handle)
    num_matches            = len(difflib.SequenceMatcher(None, real_name,user_handle).get_matching_blocks())
    total_urls_in_desc     = len(user_info._json['entities']['description']['urls'])
    official_url_exists    = expanded_url
    followers_count        = user_info._json['followers_count']
    friends_count          = user_info._json['friends_count']
    listed_count           = user_info._json['listed_count']
    favourites_count       = user_info._json['favourites_count']
    geo_enabled            = int(user_info._json['geo_enabled'])
    acc_created_on         = user_info._json['created_at'].split()
    acc_life               = calculate_days_past(acc_created_on)
    verified               = int(user_info._json['verified'])
    num_tweets             = user_info._json['statuses_count']
    protected              = int(user_info._json['protected'])
    avg_likes_per_tweet    = favourites_count/num_tweets
    latest_tweet_time      = user_info._json['status']['created_at'].split()
    activity               = calculate_days_past(latest_tweet_time)

    if(acc_life==0):
        posting_frequency = num_tweets
    else:
        posting_frequency      = num_tweets/acc_life

    
    if(friends_count!=0):
        follower_friends_ratio = followers_count/friends_count
    else:
        follower_friends_ratio = 9999999
    
    list = [tweet_text, tweet_link, desc,
            real_name, user_handle, user_id,
            chars_in_desc, chars_in_real_name,
            chars_in_user_handle, num_matches,
            total_urls_in_desc, official_url_exists,
            followers_count, friends_count,
            listed_count, favourites_count,
            geo_enabled, acc_created_on,
            acc_life, verified, num_tweets,
            protected, posting_frequency,
            latest_tweet_time, activity,
            avg_likes_per_tweet, follower_friends_ratio]
    
    # for item in list:
    #     print(item)
        
    return list

In [390]:
tweet_ids_list = df['id'].values.tolist()

In [391]:
# Creating User Features Dataset
count = 1
new_users_features_list = []

for id in range(len(tweet_ids_list)):


    try:

        tweet = api.get_status(df['id'][id],tweet_mode='extended')
        user_info = get_user_info(tweet)
        user_info.insert(0,df['id'][id])
        new_users_features_list.append(user_info)
        count = count+1

        if(count>300):
            break

    except tweepy.TweepError as e:
        print("Tweepy error : " + str(e))
        continue

print(len(new_users_features_list))

13


In [392]:
print(len(new_users_features_list))
print(count)

13
14


In [393]:
df = pd.DataFrame(new_users_features_list,columns = [ 'tweet_id', 'tweet_text', 'tweet_link', 'desc', 'real_name', 'user_handle', 'user_id', 'chars_in_desc', 'chars_in_real_name',
                                                            'chars_in_user_handle', 'num_matches', 'total_urls_in_desc', 'official_url_exists', 'followers_count',
                                                            'friends_count', 'listed_count', 'favourites_count', 'geo_enabled', 'acc_created_on', 'acc_life', 'verified',
                                                            'num_tweets', 'protected', 'posting_frequency', 'latest_tweet_time', 'activity', 'avg_likes_per_tweet',
                                                            'follower_friends_ratio'])


In [394]:
df

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio
0,1277587874100068355,"RT @PIBAgartala: #COVID_19 ২৯ জুন, ২০২০ সকাল ৮...",http://twitter.com/anyuser/status/127758787410...,"Student in Dharmanagar, Govt Degree College...",Pranendu Bhattacharjee,PranenduBhatta1,1249580478950076418,46,22,15,3,0,0,99,77,0,1751,0,"[Mon, Apr, 13, 06:09:53, +0000, 2020]",77,0,1754,0,22.779221,"[Mon, Jun, 29, 13:01:24, +0000, 2020]",0,0.998290,1.285714
1,1277585827514777600,RT @Asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,http://twitter.com/anyuser/status/127758582751...,Thanks God.,Kalipada Dandapat,KalipadaDandap3,1257296248379305984,11,17,15,3,0,0,21,8,0,220,0,"[Mon, May, 04, 13:09:36, +0000, 2020]",56,0,237,0,4.232143,"[Mon, Jun, 29, 12:53:13, +0000, 2020]",0,0.928270,2.625000
2,1277580484164243456,@Dharmayuddhaa @MamataOfficial মাথামোটা! তোর ব...,http://twitter.com/anyuser/status/127758048416...,Economics Bsc (Hons) student | R.K. Ashramian ...,Subhayan Mukherjee Tito,titomukherjee15,861548391972700161,125,23,15,2,0,1,963,1097,0,50178,1,"[Mon, May, 08, 11:48:38, +0000, 2017]",1148,0,10773,0,9.384146,"[Mon, Jun, 29, 12:38:26, +0000, 2020]",0,4.657755,0.877849
3,1277578556596383748,১৫ জুলাই অবধি বিশেষ ট্রেন চলাচল বন্ধ তামিলনাড়...,http://twitter.com/anyuser/status/127757855659...,Facebook: https://t.co/vQFs4apIJo\nYoutube Cha...,Aajkaal Official,aajkaalofficial,795241907597123584,111,16,15,3,3,1,354,134,1,11,0,"[Sun, Nov, 06, 12:30:20, +0000, 2016]",1331,0,4138,0,3.108941,"[Mon, Jun, 29, 12:23:58, +0000, 2020]",0,0.002658,2.641791
4,1277578344754630662,করোনা থেকে সেরে ওঠা ব্যক্তিদের সরকারি হাসপাতাল...,http://twitter.com/anyuser/status/127757834475...,"From politics to sports, latest news for the B...",Hindustan Times Bangla,HT_Bangla,1192400038216916992,79,22,9,4,0,1,693,21,1,2,0,"[Thu, Nov, 07, 11:15:32, +0000, 2019]",235,1,10721,0,45.621277,"[Mon, Jun, 29, 12:34:08, +0000, 2020]",0,0.000187,33.000000
5,1277578048166977536,"দিল্লীৰ নিচিনা ঠাইত শুনিব পোৱা মতে, #COVID পৰী...",http://twitter.com/anyuser/status/127757804816...,"I like traveling, sense of humor, video games,...",Angira Bharadwaj,BharadwajAngira,1058272790745206784,88,16,15,2,0,0,493,321,1,2412,0,"[Fri, Nov, 02, 08:21:09, +0000, 2018]",605,0,3921,0,6.480992,"[Mon, Jun, 29, 13:00:32, +0000, 2020]",0,0.615149,1.535826
6,1277578021000450048,"COVID যোদ্ধাদের কুর্নিশ রাজ্য সরকারের, ১ জুলাই...",http://twitter.com/anyuser/status/127757802100...,I am determined to be cheerful & happy in what...,ARIJIT KR. ROY🇮🇳😷,IamArijitRoy,89647717,151,17,12,3,0,0,104,567,4,13434,0,"[Fri, Nov, 13, 07:17:25, +0000, 2009]",3881,0,27389,0,7.057202,"[Mon, Jun, 29, 12:55:14, +0000, 2020]",0,0.490489,0.183422
7,1277577535216250880,এবার স্বাস্থ্য পরিসেবায় সরাসরি অংশগ্রহণ করবেন...,http://twitter.com/anyuser/status/127757753521...,General Secretary of Surendranath Law College ...,অমিতাভ || Amitava,AdvAmitavaAich,1272195565632262144,157,17,14,2,0,0,88,233,0,646,0,"[Sun, Jun, 14, 15:54:07, +0000, 2020]",15,0,93,0,6.200000,"[Mon, Jun, 29, 12:23:00, +0000, 2020]",0,6.946237,0.377682
8,1277575323387715584,মৰিগাঁৱত পেট্ৰল পাম্পৰ কৰ্মচাৰীৰ দেহত কোৰোনা ভ...,http://twitter.com/anyuser/status/127757532338...,LIGHTNING FAST ALEARTS #BREAKINGNEWS FROM ASSA...,asomiyapratidin,pratidinasomiya,2241832560,104,15,15,2,0,1,15016,267,40,0,0,"[Thu, Dec, 12, 05:13:53, +0000, 2013]",2391,0,32688,0,13.671267,"[Mon, Jun, 29, 12:43:50, +0000, 2020]",0,0.000000,56.239700
9,1277573897748189185,চিকিৎসাধীন কোভিড-১৯ সংক্রমিতদের থেকে ১ লক্ষর ব...,http://twitter.com/anyuser/status/127757389774...,"Raina Bidhansabha, Mondal 8,Vice President & I...",Chowkidar Asim Dhaure,asim_dhaure,749303887635574784,86,21,11,4,0,0,1

In [395]:
def preprocess_tweet(tweet):

    tweet = tweet.lower()
    url = r'http\S+'
    tweet = re.sub(url, 'URL', tweet, flags=re.MULTILINE)
    emoji = re.compile("["         u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002500-\U00002BEF"  # chinese char
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
    tweet =  emoji.sub(r'', tweet)
    tweet = ' '.join([word[1:] if word[0] == '#' else word for word in tweet.split()])
    
    return tweet

In [396]:
pip install googletrans

In [397]:
from googletrans import Translator
translator = Translator()

In [398]:
import re

for i in range(len(df)):
    df.at[i,'tweet_text'] = preprocess_tweet(df.iloc[i]['tweet_text'])

In [399]:
def too_much_cleaned_tweet(tweet):

    
    cleaned_text = tweet.split()
    exclude_char_list = ['#','@','rt','|','%', '[', '(', '{', '}', ']', ')','url','URL','covid','covid19','corona','coronavirus','0','1','2','3','4','5','6','7','8','9']
    tweet_without_special_chars = [word for word in cleaned_text if all(ch not in word for ch in exclude_char_list)]
    tweet_without_special_chars = ' '.join(tweet_without_special_chars)
 
    return tweet_without_special_chars

    


In [400]:
list = []
for tweet_text in df['tweet_text']:
    list.append(too_much_cleaned_tweet(tweet_text))

cleaned_tweet_df = pd.DataFrame(list,columns = ['too_much_cleaned_tweet'])
cleaned_tweet_df

,too_much_cleaned_tweet
0,"২৯ জুন, ২০২০ সকাল ৮. টা পর্যন্ত ত্রিপুরায় কোভ..."
1,নয়াগ্রাম ব্লক তৃণমূল কংগ্রেসের সম্মানীয় সভাপ...
2,মাথামোটা! তোর বাপ দিয়েছিল দেখ।
3,১৫ জুলাই অবধি বিশেষ ট্রেন চলাচল বন্ধ তামিলনাড়ুতে
4,করোনা থেকে সেরে ওঠা ব্যক্তিদের সরকারি হাসপাতাল...
5,"দিল্লীৰ নিচিনা ঠাইত শুনিব পোৱা মতে, পৰীক্ষা কৰ..."
6,"যোদ্ধাদের কুর্নিশ রাজ্য সরকারের, ১ জুলাই রাজ্য..."
7,এবার স্বাস্থ্য পরিসেবায় সরাসরি অংশগ্রহণ করবেন...
8,মৰিগাঁৱত পেট্ৰল পাম্পৰ কৰ্মচাৰীৰ দেহত কোৰোনা ভ...
9,চিকিৎসাধীন কোভিড-১৯ সংক্রমিতদের থেকে ১ লক্ষর ব...


In [401]:
df = pd.concat([df,cleaned_tweet_df],axis=1)
df

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio,too_much_cleaned_tweet
0,1277587874100068355,"rt @pibagartala: covid_19 ২৯ জুন, ২০২০ সকাল ৮....",http://twitter.com/anyuser/status/127758787410...,"Student in Dharmanagar, Govt Degree College...",Pranendu Bhattacharjee,PranenduBhatta1,1249580478950076418,46,22,15,3,0,0,99,77,0,1751,0,"[Mon, Apr, 13, 06:09:53, +0000, 2020]",77,0,1754,0,22.779221,"[Mon, Jun, 29, 13:01:24, +0000, 2020]",0,0.998290,1.285714,"২৯ জুন, ২০২০ সকাল ৮. টা পর্যন্ত ত্রিপুরায় কোভ..."
1,1277585827514777600,rt @asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,http://twitter.com/anyuser/status/127758582751...,Thanks God.,Kalipada Dandapat,KalipadaDandap3,1257296248379305984,11,17,15,3,0,0,21,8,0,220,0,"[Mon, May, 04, 13:09:36, +0000, 2020]",56,0,237,0,4.232143,"[Mon, Jun, 29, 12:53:13, +0000, 2020]",0,0.928270,2.625000,নয়াগ্রাম ব্লক তৃণমূল কংগ্রেসের সম্মানীয় সভাপ...
2,1277580484164243456,@dharmayuddhaa @mamataofficial মাথামোটা! তোর ব...,http://twitter.com/anyuser/status/127758048416...,Economics Bsc (Hons) student | R.K. Ashramian ...,Subhayan Mukherjee Tito,titomukherjee15,861548391972700161,125,23,15,2,0,1,963,1097,0,50178,1,"[Mon, May, 08, 11:48:38, +0000, 2017]",1148,0,10773,0,9.384146,"[Mon, Jun, 29, 12:38:26, +0000, 2020]",0,4.657755,0.877849,মাথামোটা! তোর বাপ দিয়েছিল দেখ।
3,1277578556596383748,১৫ জুলাই অবধি বিশেষ ট্রেন চলাচল বন্ধ তামিলনাড়...,http://twitter.com/anyuser/status/127757855659...,Facebook: https://t.co/vQFs4apIJo\nYoutube Cha...,Aajkaal Official,aajkaalofficial,795241907597123584,111,16,15,3,3,1,354,134,1,11,0,"[Sun, Nov, 06, 12:30:20, +0000, 2016]",1331,0,4138,0,3.108941,"[Mon, Jun, 29, 12:23:58, +0000, 2020]",0,0.002658,2.641791,১৫ জুলাই অবধি বিশেষ ট্রেন চলাচল বন্ধ তামিলনাড়ুতে
4,1277578344754630662,করোনা থেকে সেরে ওঠা ব্যক্তিদের সরকারি হাসপাতাল...,http://twitter.com/anyuser/status/127757834475...,"From politics to sports, latest news for the B...",Hindustan Times Bangla,HT_Bangla,1192400038216916992,79,22,9,4,0,1,693,21,1,2,0,"[Thu, Nov, 07, 11:15:32, +0000, 2019]",235,1,10721,0,45.621277,"[Mon, Jun, 29, 12:34:08, +0000, 2020]",0,0.000187,33.000000,করোনা থেকে সেরে ওঠা ব্যক্তিদের সরকারি হাসপাতাল...
5,1277578048166977536,"দিল্লীৰ নিচিনা ঠাইত শুনিব পোৱা মতে, covid পৰীক...",http://twitter.com/anyuser/status/127757804816...,"I like traveling, sense of humor, video games,...",Angira Bharadwaj,BharadwajAngira,1058272790745206784,88,16,15,2,0,0,493,321,1,2412,0,"[Fri, Nov, 02, 08:21:09, +0000, 2018]",605,0,3921,0,6.480992,"[Mon, Jun, 29, 13:00:32, +0000, 2020]",0,0.615149,1.535826,"দিল্লীৰ নিচিনা ঠাইত শুনিব পোৱা মতে, পৰীক্ষা কৰ..."
6,1277578021000450048,"covid যোদ্ধাদের কুর্নিশ রাজ্য সরকারের, ১ জুলাই...",http://twitter.com/anyuser/status/127757802100...,I am determined to be cheerful & happy in what...,ARIJIT KR. ROY🇮🇳😷,IamArijitRoy,89647717,151,17,12,3,0,0,104,567,4,13434,0,"[Fri, Nov, 13, 07:17:25, +0000, 2009]",3881,0,27389,0,7.057202,"[Mon, Jun, 29, 12:55:14, +0000, 2020]",0,0.490489,0.183422,"যোদ্ধাদের কুর্নিশ রাজ্য সরকারের, ১ জুলাই রাজ্য..."
7,1277577535216250880,এবার স্বাস্থ্য পরিসেবায় সরাসরি অংশগ্রহণ করবেন...,http://twitter.com/anyuser/status/127757753521...,General Secretary of Surendranath Law College ...,অমিতাভ || Amitava,AdvAmitavaAich,1272195565632262144,157,17,14,2,0,0,88,233,0,646,0,"[Sun, Jun, 14, 15:54:07, +0000, 2020]",15,0,93,0,6.200000,"[Mon, Jun, 29, 12:23:00, +0000, 2020]",0,6.946237,0.377682,এবার স্বাস্থ্য পরিসেবায় সরাসরি অংশগ্রহণ করবেন...
8,1277575323387715584,মৰিগাঁৱত পেট্ৰল পাম্পৰ কৰ্মচাৰীৰ দেহত কোৰোনা ভ...,http://twitter.com/anyuser/status/127757532338...,LIGHTNING FAST ALEARTS #BREAKINGNEWS FROM ASSA...,asomiyapratidin,pratidinasomiya,22

In [402]:
idx_drop = [index for index, row in df.iterrows() if translator.detect(row['too_much_cleaned_tweet']).lang == 'en']
idx_drop

[]

In [403]:
df_modified = copy.deepcopy(df)
df_modified = df_modified.drop(idx_drop)
df_modified = df_modified.drop(['too_much_cleaned_tweet'],axis=1)
df_modified

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio
0,1277587874100068355,"rt @pibagartala: covid_19 ২৯ জুন, ২০২০ সকাল ৮....",http://twitter.com/anyuser/status/127758787410...,"Student in Dharmanagar, Govt Degree College...",Pranendu Bhattacharjee,PranenduBhatta1,1249580478950076418,46,22,15,3,0,0,99,77,0,1751,0,"[Mon, Apr, 13, 06:09:53, +0000, 2020]",77,0,1754,0,22.779221,"[Mon, Jun, 29, 13:01:24, +0000, 2020]",0,0.998290,1.285714
1,1277585827514777600,rt @asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,http://twitter.com/anyuser/status/127758582751...,Thanks God.,Kalipada Dandapat,KalipadaDandap3,1257296248379305984,11,17,15,3,0,0,21,8,0,220,0,"[Mon, May, 04, 13:09:36, +0000, 2020]",56,0,237,0,4.232143,"[Mon, Jun, 29, 12:53:13, +0000, 2020]",0,0.928270,2.625000
2,1277580484164243456,@dharmayuddhaa @mamataofficial মাথামোটা! তোর ব...,http://twitter.com/anyuser/status/127758048416...,Economics Bsc (Hons) student | R.K. Ashramian ...,Subhayan Mukherjee Tito,titomukherjee15,861548391972700161,125,23,15,2,0,1,963,1097,0,50178,1,"[Mon, May, 08, 11:48:38, +0000, 2017]",1148,0,10773,0,9.384146,"[Mon, Jun, 29, 12:38:26, +0000, 2020]",0,4.657755,0.877849
3,1277578556596383748,১৫ জুলাই অবধি বিশেষ ট্রেন চলাচল বন্ধ তামিলনাড়...,http://twitter.com/anyuser/status/127757855659...,Facebook: https://t.co/vQFs4apIJo\nYoutube Cha...,Aajkaal Official,aajkaalofficial,795241907597123584,111,16,15,3,3,1,354,134,1,11,0,"[Sun, Nov, 06, 12:30:20, +0000, 2016]",1331,0,4138,0,3.108941,"[Mon, Jun, 29, 12:23:58, +0000, 2020]",0,0.002658,2.641791
4,1277578344754630662,করোনা থেকে সেরে ওঠা ব্যক্তিদের সরকারি হাসপাতাল...,http://twitter.com/anyuser/status/127757834475...,"From politics to sports, latest news for the B...",Hindustan Times Bangla,HT_Bangla,1192400038216916992,79,22,9,4,0,1,693,21,1,2,0,"[Thu, Nov, 07, 11:15:32, +0000, 2019]",235,1,10721,0,45.621277,"[Mon, Jun, 29, 12:34:08, +0000, 2020]",0,0.000187,33.000000
5,1277578048166977536,"দিল্লীৰ নিচিনা ঠাইত শুনিব পোৱা মতে, covid পৰীক...",http://twitter.com/anyuser/status/127757804816...,"I like traveling, sense of humor, video games,...",Angira Bharadwaj,BharadwajAngira,1058272790745206784,88,16,15,2,0,0,493,321,1,2412,0,"[Fri, Nov, 02, 08:21:09, +0000, 2018]",605,0,3921,0,6.480992,"[Mon, Jun, 29, 13:00:32, +0000, 2020]",0,0.615149,1.535826
6,1277578021000450048,"covid যোদ্ধাদের কুর্নিশ রাজ্য সরকারের, ১ জুলাই...",http://twitter.com/anyuser/status/127757802100...,I am determined to be cheerful & happy in what...,ARIJIT KR. ROY🇮🇳😷,IamArijitRoy,89647717,151,17,12,3,0,0,104,567,4,13434,0,"[Fri, Nov, 13, 07:17:25, +0000, 2009]",3881,0,27389,0,7.057202,"[Mon, Jun, 29, 12:55:14, +0000, 2020]",0,0.490489,0.183422
7,1277577535216250880,এবার স্বাস্থ্য পরিসেবায় সরাসরি অংশগ্রহণ করবেন...,http://twitter.com/anyuser/status/127757753521...,General Secretary of Surendranath Law College ...,অমিতাভ || Amitava,AdvAmitavaAich,1272195565632262144,157,17,14,2,0,0,88,233,0,646,0,"[Sun, Jun, 14, 15:54:07, +0000, 2020]",15,0,93,0,6.200000,"[Mon, Jun, 29, 12:23:00, +0000, 2020]",0,6.946237,0.377682
8,1277575323387715584,মৰিগাঁৱত পেট্ৰল পাম্পৰ কৰ্মচাৰীৰ দেহত কোৰোনা ভ...,http://twitter.com/anyuser/status/127757532338...,LIGHTNING FAST ALEARTS #BREAKINGNEWS FROM ASSA...,asomiyapratidin,pratidinasomiya,2241832560,104,15,15,2,0,1,15016,267,40,0,0,"[Thu, Dec, 12, 05:13:53, +0000, 2013]",2391,0,32688,0,13.671267,"[Mon, Jun, 29, 12:43:50, +0000, 2020]",0,0.000000,56.239700
9,1277573897748189185,চিকিৎসাধীন কোভিড-১৯ সংক্রমিতদের থেকে ১ লক্ষর ব...,http://twitter.com/anyuser/status/127757389774...,"Raina Bidhansabha, Mondal 8,Vice President & I...",Chowkidar Asim Dhaure,asim_dhaure,749303887635574784,86,21,11,4,0,0,1

In [404]:
### Adding Label's field --> (( 1=Real || 2=Fake || 3=Maybe ))

df['label'] = pd.DataFrame((np.ones(len(df),dtype=int)*3))
df

,tweet_id,tweet_text,tweet_link,desc,real_name,user_handle,user_id,chars_in_desc,chars_in_real_name,chars_in_user_handle,num_matches,total_urls_in_desc,official_url_exists,followers_count,friends_count,listed_count,favourites_count,geo_enabled,acc_created_on,acc_life,verified,num_tweets,protected,posting_frequency,latest_tweet_time,activity,avg_likes_per_tweet,follower_friends_ratio,too_much_cleaned_tweet,label
0,1277587874100068355,"rt @pibagartala: covid_19 ২৯ জুন, ২০২০ সকাল ৮....",http://twitter.com/anyuser/status/127758787410...,"Student in Dharmanagar, Govt Degree College...",Pranendu Bhattacharjee,PranenduBhatta1,1249580478950076418,46,22,15,3,0,0,99,77,0,1751,0,"[Mon, Apr, 13, 06:09:53, +0000, 2020]",77,0,1754,0,22.779221,"[Mon, Jun, 29, 13:01:24, +0000, 2020]",0,0.998290,1.285714,"২৯ জুন, ২০২০ সকাল ৮. টা পর্যন্ত ত্রিপুরায় কোভ...",3
1,1277585827514777600,rt @asishsahoo12341: নয়াগ্রাম ব্লক তৃণমূল কংগ...,http://twitter.com/anyuser/status/127758582751...,Thanks God.,Kalipada Dandapat,KalipadaDandap3,1257296248379305984,11,17,15,3,0,0,21,8,0,220,0,"[Mon, May, 04, 13:09:36, +0000, 2020]",56,0,237,0,4.232143,"[Mon, Jun, 29, 12:53:13, +0000, 2020]",0,0.928270,2.625000,নয়াগ্রাম ব্লক তৃণমূল কংগ্রেসের সম্মানীয় সভাপ...,3
2,1277580484164243456,@dharmayuddhaa @mamataofficial মাথামোটা! তোর ব...,http://twitter.com/anyuser/status/127758048416...,Economics Bsc (Hons) student | R.K. Ashramian ...,Subhayan Mukherjee Tito,titomukherjee15,861548391972700161,125,23,15,2,0,1,963,1097,0,50178,1,"[Mon, May, 08, 11:48:38, +0000, 2017]",1148,0,10773,0,9.384146,"[Mon, Jun, 29, 12:38:26, +0000, 2020]",0,4.657755,0.877849,মাথামোটা! তোর বাপ দিয়েছিল দেখ।,3
3,1277578556596383748,১৫ জুলাই অবধি বিশেষ ট্রেন চলাচল বন্ধ তামিলনাড়...,http://twitter.com/anyuser/status/127757855659...,Facebook: https://t.co/vQFs4apIJo\nYoutube Cha...,Aajkaal Official,aajkaalofficial,795241907597123584,111,16,15,3,3,1,354,134,1,11,0,"[Sun, Nov, 06, 12:30:20, +0000, 2016]",1331,0,4138,0,3.108941,"[Mon, Jun, 29, 12:23:58, +0000, 2020]",0,0.002658,2.641791,১৫ জুলাই অবধি বিশেষ ট্রেন চলাচল বন্ধ তামিলনাড়ুতে,3
4,1277578344754630662,করোনা থেকে সেরে ওঠা ব্যক্তিদের সরকারি হাসপাতাল...,http://twitter.com/anyuser/status/127757834475...,"From politics to sports, latest news for the B...",Hindustan Times Bangla,HT_Bangla,1192400038216916992,79,22,9,4,0,1,693,21,1,2,0,"[Thu, Nov, 07, 11:15:32, +0000, 2019]",235,1,10721,0,45.621277,"[Mon, Jun, 29, 12:34:08, +0000, 2020]",0,0.000187,33.000000,করোনা থেকে সেরে ওঠা ব্যক্তিদের সরকারি হাসপাতাল...,3
5,1277578048166977536,"দিল্লীৰ নিচিনা ঠাইত শুনিব পোৱা মতে, covid পৰীক...",http://twitter.com/anyuser/status/127757804816...,"I like traveling, sense of humor, video games,...",Angira Bharadwaj,BharadwajAngira,1058272790745206784,88,16,15,2,0,0,493,321,1,2412,0,"[Fri, Nov, 02, 08:21:09, +0000, 2018]",605,0,3921,0,6.480992,"[Mon, Jun, 29, 13:00:32, +0000, 2020]",0,0.615149,1.535826,"দিল্লীৰ নিচিনা ঠাইত শুনিব পোৱা মতে, পৰীক্ষা কৰ...",3
6,1277578021000450048,"covid যোদ্ধাদের কুর্নিশ রাজ্য সরকারের, ১ জুলাই...",http://twitter.com/anyuser/status/127757802100...,I am determined to be cheerful & happy in what...,ARIJIT KR. ROY🇮🇳😷,IamArijitRoy,89647717,151,17,12,3,0,0,104,567,4,13434,0,"[Fri, Nov, 13, 07:17:25, +0000, 2009]",3881,0,27389,0,7.057202,"[Mon, Jun, 29, 12:55:14, +0000, 2020]",0,0.490489,0.183422,"যোদ্ধাদের কুর্নিশ রাজ্য সরকারের, ১ জুলাই রাজ্য...",3
7,1277577535216250880,এবার স্বাস্থ্য পরিসেবায় সরাসরি অংশগ্রহণ করবেন...,http://twitter.com/anyuser/status/127757753521...,General Secretary of Surendranath Law College ...,অমিতাভ || Amitava,AdvAmitavaAich,1272195565632262144,157,17,14,2,0,0,88,233,0,646,0,"[Sun, Jun, 14, 15:54:07, +0000, 2020]",15,0,93,0,6.200000,"[Mon, Jun, 29, 12:23:00, +0000, 2020]",0,6.946237,0.377682,এবার স্বাস্থ্য পরিসেবায় সরাসরি অংশগ্রহণ করবেন...,3
8,1277575323387715584,মৰিগাঁৱত পেট্ৰল পাম্পৰ কৰ্মচাৰীৰ দেহত কোৰোনা ভ...,http://twitter.com/anyuser/status/127757532338...,LIGHTNING FAST ALEARTS #BREAKINGNEWS FROM ASSA...,asomiyaprati

In [405]:
df.to_csv("/gdrive/My Drive/2nd Sem/IBM/Covid Project/Dataset/Scraping Tweets Demo/covid_tweets_with_user_features.csv",index=False)